In [ ]:
import sys
import re
from IPython.display import clear_output

sys.path.append("./matlab/myBTD2/myBTD2/for_redistribution_files_only")

import os

os.environ["LD_LIBRARY_PATH"] = "/home/renan/MATLAB/R2021b/runtime/glnxa64/"

import myBTD2
import numpy as np
import warnings
from tqdm import tqdm
import matlab
from scipy.linalg import hankel


warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning)


def hankelization(recording):
    _, n_lead = recording.shape

    T = []
    for i in range(0, n_lead):
        vec = recording[:, i]
        vec_filtered_hankel = hankel(vec)
        T.append(vec_filtered_hankel)

    T = np.array(T)
    T = matlab.double(T.tolist(), size=T.shape)

    return T


def get_btd(recording):
    btd = myBTD2.initialize()
    _, output = btd.myBTD2(
        5,
        matlab.double([17], size=(1, 1)),
        matlab.double([300], size=(1, 1)),
        matlab.double([1.0e-5], size=(1, 1)),
        matlab.double([1.0e-5], size=(1, 1)),
        matlab.double([False], size=(1, 1)),
        hankelization(recording),
        nargout=2,
    )
    btd.terminate()

    keys_to_extract = [
        "iter",
        "fval",
        "relfval",
        "relstep",
        "delta",
        "rho",
        "relerr",
    ]

    object_output = [output[key] for key in keys_to_extract]

    return object_output

## Extract the Histogram+BTD from the Chunks Data

In [2]:
alias_db = "LTAFDB"

source_files = [
    filename
    for filename in os.listdir(f"../extract_chunks/{alias_db}")
    if re.search(r"_0", filename)
]

chunks_remains = sorted(
    [
        file
        for file in source_files
        if file not in os.listdir(f"./50_bins_hist_btd/{alias_db}")
    ]
)

In [14]:
for filename in chunks_remains:
    samples = np.load(f"../extract_chunks/{alias_db}/{filename}", allow_pickle=True)

    recording_result = np.empty((0, 57))

    for sample in tqdm(samples):
        rri_segment, recording = sample

        try:
            rri_histogram, _ = np.histogram(a=rri_segment, range=(0, 2500), bins=50)

            if rri_histogram.shape[0] == 0:
                continue

            btd = get_btd(recording)
            clear_output(wait=True)

            combine = np.hstack((rri_histogram, np.array(btd)))

            recording_result = np.vstack((recording_result, combine))

        except Exception as e:
            print(filename, e)

    np.save(
        file=f"./50_bins_hist_btd/{alias_db}/{filename}",
        arr=recording_result,
    )

  0%|          | 0/200 [00:00<?, ?it/s]

> In ll1_core/state (line 237)
In ll1_core (line 191)
In ll1_nls (line 40)
In myBTD2 (line 9)

> In ll1_core/state (line 237)
In ll1_core/grad (line 256)
In nls_gndl (line 369)
In ll1_core (line 193)
In ll1_nls (line 40)
In myBTD2 (line 9)

> In ll1_core/state (line 237)
In ll1_core/grad (line 256)
In nls_gndl (line 369)
In ll1_core (line 193)
In ll1_nls (line 40)
In myBTD2 (line 9)

> In ll1_core/state (line 237)
In ll1_core/grad (line 256)
In nls_gndl (line 369)
In ll1_core (line 193)
In ll1_nls (line 40)
In myBTD2 (line 9)

> In ll1_core/state (line 237)
In ll1_core/grad (line 256)
In nls_gndl (line 369)
In ll1_core (line 193)
In ll1_nls (line 40)
In myBTD2 (line 9)

> In ll1_core/state (line 237)
In ll1_core/grad (line 256)
In nls_gndl (line 369)
In ll1_core (line 193)
In ll1_nls (line 40)
In myBTD2 (line 9)

> In ll1_core/state (line 237)
In ll1_core/grad (line 256)
In nls_gndl (line 369)
In ll1_core (line 193)
In ll1_nls (line 40)
In myBTD2 (line 9)

> In ll1_core/state (line 237

  0%|          | 1/200 [00:23<1:16:50, 23.17s/it]

The command was cancelled. 
105_0_200_result.npy cannot unpack non-iterable bool object


## Extract the Histograms from the Chunks Data

In [ ]:
import numpy as np
import os
from tqdm import tqdm

ALIAS_DB = ["AFDB", "NSRDB", "LTAFDB", "LTAFDB_N"]
BINS = 10, 25, 50

for bins in BINS:
    for alias in ALIAS_DB:
        source_files = [
            filename for filename in os.listdir(f"../extract_chunks/{alias}")
        ]

        for filename in source_files:
            samples = np.load(
                f"../extract_chunks/{alias}/{filename}", allow_pickle=True
            )

            recording_result = np.empty((0, bins))

            for sample in tqdm(samples):
                rri_segment, recording = sample

                rri_histogram, _ = np.histogram(
                    a=rri_segment, range=(0, 2500), bins=bins
                )

                if rri_histogram.shape[0] == 0:
                    continue

                recording_result = np.vstack((recording_result, rri_histogram))

            save_path = f"./{bins}_bins_only_hist/{alias}/"

            if not os.path.exists(save_path):
                os.makedirs(save_path)

            np.save(
                file=f"{save_path}/{filename}",
                arr=recording_result,
            )

## Combine the RRIs + Extracted BTD

In [ ]:
import numpy as np
import os

ALIAS_DB = ["AFDB", "NSRDB", "LTAFDB", "LTAFDB_N"]

for alias in ALIAS_DB:
    source_files = [filename for filename in os.listdir(f"./50_bins_hist_btd/{alias}")]

    for filename in source_files:
        hist_btd_samples = np.load(
            f"./50_bins_hist_btd/{alias}/{filename}", allow_pickle=True
        )

        btd_samples = hist_btd_samples[:, -7:]

        chunks_sample = np.load(
            os.path.join(f"../extract_chunks/{alias}", filename), allow_pickle=True
        )

        rri_samples = np.vstack(chunks_sample[:, 0])

        samples = np.hstack((rri_samples, btd_samples))

        save_path = f"./rri_btd/{alias}/"

        if not os.path.exists(save_path):
            os.makedirs(save_path)

        np.save(
            file=f"{save_path}/{filename}",
            arr=samples,
        )